In [1]:
from pymongo import MongoClient
import pandas as pd
from datetime import datetime

client = MongoClient('sandlet')
db = client.reality
coll = db['sreality']

In [2]:
dataset = coll.aggregate([
    {'$match': {
        'timeAdded': {'$gt': datetime.strptime('2020-03-17', '%Y-%m-%d')}
    }},
    {'$group': {
        '_id': "$hash_id",
        'labelsAll': {'$first': "$labelsAll"},
        'price': {'$first': "$price"},
        'name': {'$first': "$name"},
        'locality': {'$first': "$seo.locality"},
        'public_transport_distance': {'$last': "$closestPublicTransportStop.distance"}
    }}
 ])
df_original = pd.DataFrame(dataset)

In [3]:
from reality_prepare_rent_dataset import prepareDataset
cols = pd.read_csv('columns.csv', header = None).iloc[:, 0].tolist()
df = prepareDataset(df_original.copy())

In [4]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [5]:
X_p = df.drop(columns = "price_per_sq_meter")
pca = PCA()
m = len(X_p)
scaled_feature = StandardScaler().fit_transform(np.asarray(X_train['public_transport_distance']).reshape(-1,1))
X_trans = X_p
X_trans['public_transport_distance'] = scaled_feature
X_trans = pca.fit_transform(X_trans)
X_red = X_trans[:, 0:92]

In [6]:
import joblib
model = joblib.load('rent-model.huber')
price_per_sq_meter = model.predict(X_red)
df_original['price_per_sq_meter'] = price_per_sq_meter

In [7]:
df_original

,_id,labelsAll,price,name,locality,public_transport_distance,price_per_sq_meter
0,442523996,"[[personal, balcony, brick, cellar, elevator, ...",5900000,Prodej bytu 1+kk 44 m²,praha-vinohrady-u-zvonarky,88.880385,443.578713
1,1385021020,"[[personal, after_reconstruction, balcony, bri...",13490000,Prodej bytu 3+kk 95 m²,praha-stare-mesto-krocinova,114.722363,351.463633
2,1929395804,"[[personal, brick, elevator], [candy_shop, mov...",4900000,Prodej bytu 2+kk 57 m²,praha-nusle-u-pernstejnskych,184.171402,422.311467
3,3431456348,"[[personal, brick, elevator, furnished], [cand...",4400000,Prodej bytu 2+kk 54 m²,praha-zabehlice-mattioliho,423.106409,374.188287
4,1403440732,"[[personal, balcony, elevator], [candy_shop, m...",3790000,Prodej bytu 1+1 37 m²,praha-vrsovice-ruska,331.033178,343.583217
...,...,...,...,...,...,...,...
4125,3307990620,"[[collective, after_reconstruction, loggia, pa...",4690000,Prodej bytu 3+kk 66 m²,praha-horni-mecholupy-janovska,2474.079309,313.649604
4126,1106329180,"[[personal, loggia, brick, elevator, parking_l...",9950000,Prodej bytu 3+kk 85 m²,praha-chodov-starochodovska,431.279790,223.747467
4127,1049292380,"[[new_building, state, terrace, elevator], [ca...",1,Prodej bytu 4+kk 173 m²,praha-praha-5-holeckova,196.937921,208.119197
4128,3632914012,"[[collective, brick, cellar, partly_furnished]...",7990000,Prodej bytu 3+1 96 m²,praha-karlin-saldova,158.365799,234.890544
